### import relevant libraries, reading data from csv, taking a look

In [27]:
import os
import numpy as np
import pandas as pd
from pandasql import sqldf 
from matplotlib import pyplot as plt
import seaborn as sns

In [4]:
# read contents in pwd
os.listdir('.')

['exploring-test-csv.ipynb',
 'problem1.ipynb',
 'exploratory-analysis.ipynb',
 'test.csv',
 'Candidate?s Hiring Pack. Product Analytics @ Bolt.pdf',
 '.ipynb_checkpoints',
 '.git']

In [5]:
dataset = pd.read_csv('test.csv')

In [6]:
dataset.head(2)

,order_id_new,order_try_id_new,calc_created,metered_price,upfront_price,distance,duration,gps_confidence,entered_by,b_state,...,device_token,rider_app_version,order_state,order_try_state,driver_app_version,driver_device_uid_new,device_name,eu_indicator,overpaid_ride_ticket,fraud_score
0,22,22,2020-02-02 3:37:31,4.04,10.0,2839,700,1,client,finished,...,NaN,CI.4.17,finished,finished,DA.4.37,1596,Xiaomi Redmi 6,1,0,-1383.0
1,618,618,2020-02-08 2:26:19,6.09,3.6,5698,493,1,client,finished,...,NaN,CA.5.43,finished,finished,DA.4.39,1578,Samsung SM-G965F,1,0,NaN


In [150]:
dataset[dataset.order_id_new==0]

,order_id_new,order_try_id_new,calc_created,metered_price,upfront_price,distance,duration,gps_confidence,entered_by,b_state,dest_change_number,prediction_price_type,predicted_distance,predicted_duration,change_reason_pricing,ticket_id_new,device_token,rider_app_version,order_state,order_try_state,driver_app_version,driver_device_uid_new,device_name,eu_indicator,overpaid_ride_ticket,fraud_score
1748,0,0,2020-02-02 0:01:16,7.53,4.5,6153,1194,1,client,finished,1,upfront,4379.0,480.0,NaN,1370,NaN,CA.5.42,finished,finished,DA.4.30,637,HMD Global TA-1032,1,0,NaN


In [17]:
print(f'shape : {dataset.shape}')
print(f'unique order ids new : {dataset.order_id_new.nunique()}')
print(f'unique order try id new : {dataset.order_try_id_new.nunique()}')
print(f'order_id mismatches : {len(dataset[dataset.order_id_new!=dataset.order_try_id_new].index)}')

shape : (4943, 26)
unique order ids new : 4166
unique order try id new : 4166
order_id mismatches : 58


there are more rows that there are orderIds, 

`inference` : orderIDs are not unique in the dataset shared, and there are 58 rows where the 2 order id feilds have different values

to be safe, lets check for duplicate rows

In [12]:
dataset.drop_duplicates(subset=None, inplace=True)
print(f'total rows : {len(dataset.index)}')

total rows : 4943


okay, as expected, there are no duplicate rows

lets take a look at what the duplicate rows look like

In [34]:
query = """

with base as (

select *,
count(*) over (partition by order_id_new) as total_rows_for_this_order,
count(order_id_new) over () as total_rows
from dataset
)

select *, 
case when order_id_new <> order_try_id_new then 1 else 0 end as order_id_mismatch_flag
from base 

"""

In [36]:
updated_dataset = sqldf(query)

In [37]:
duplicate_orders_df = updated_dataset[updated_dataset.total_rows_for_this_order>1].copy(deep=True)
pd.set_option('display.max_columns', None)
duplicate_orders_df.head(10)

,order_id_new,order_try_id_new,calc_created,metered_price,upfront_price,distance,duration,gps_confidence,entered_by,b_state,dest_change_number,prediction_price_type,predicted_distance,predicted_duration,change_reason_pricing,ticket_id_new,device_token,rider_app_version,order_state,order_try_state,driver_app_version,driver_device_uid_new,device_name,eu_indicator,overpaid_ride_ticket,fraud_score,total_rows_for_this_order,total_rows,order_id_mismatch_flag
3,3,3,2020-02-02 0:49:24,14.87,NaN,15541,1690,0,client,finished,2,upfront_destination_changed,5647.0,697.0,client_destination_changed,1403,None,CI.4.17,finished,finished,DA.4.37,1257,Samsung SM-J415FN,1,0,-1516.0,2,4943,0
4,3,3,2020-02-02 0:49:24,14.87,NaN,15541,1690,0,client,finished,2,upfront_destination_changed,5647.0,697.0,client_destination_changed,1434,None,CI.4.17,finished,finished,DA.4.37,1257,Samsung SM-J415FN,1,0,-1516.0,2,4943,0
14,13,13,2020-02-02 2:31:56,8.89,NaN,16880,1339,1,driver,finished,2,upfront_destination_changed,12178.0,980.0,driver_destination_changed,1481,None,CI.4.17,finished,finished,DA.4.31,858,HUAWEI P7-L10,1,0,-196.0,2,4943,0
15,13,13,2020-02-02 2:31:56,8.89,NaN,16880,1339,1,driver,finished,2,upfront_destination_changed,12178.0,980.0,driver_destination_changed,1404,None,CI.4.17,finished,finished,DA.4.31,858,HUAWEI P7-L10,1,0,-196.0,2,4943,0
21,19,19,2020-02-02 2:59:48,6.69,5.3,9498,793,1,client,finished,1,upfront,6649.0,707.0,None,1761,None,CA.5.42,finished,finished,DA.4.37,301,Xiaomi Redmi Note 7,1,0,-316.0,2,4943,0
22,19,19,2020-02-02 2:59:48,6.69,5.3,9498,793,1,client,finished,1,upfront,6649.0,707.0,None,1710,None,CA.5.42,finished,finished,DA.4.37,301,Xiaomi Redmi Note 7,1,0,-316.0,2,4943,0
29,26,26,2020-02-02 6:44:20,6000.00,8500.0,12,160,1,client,finished,1,upfront,5776.0,872.0,None,1428,None,CA.5.36,finished,finished,DA.4.37,1002,TECNO MOBILE LIMITED TECNO KC8,0,0,NaN,3,4943,0
30,26,26,2020-02-02 6:44:20,6000.00,8500.0,12,160,1,client,finished,1,upfront,5776.0,872.0,None,1382,None,CA.5.36,finished,finished,DA.4.37,1002,TECNO MOBILE LIMITED TECNO KC8,0,0,NaN,3,4943,0
31,26,26,2020-02-02 6:44:20,6000.00,8500.0,12,160,1,client,finished,1,upfront,5776.0,872.0,None,1381,None,CA.5.36,finished,finished,DA.4.37,1002,TECNO MOBILE LIMITED TECNO KC8,0,0,NaN,3,4943,0
34,29,29,2020-02-02 5:57:43,7940.22,7500.0,4869,1122,1,client,finished,1,upfront,5063.0,676.0,None,1384,None,CA.5.42,finished,finished,DA.4.31,269,Samsung SM-N920T,0,0,NaN,2,4943,0


In [48]:
print(f'ordrs with multiple rows : {duplicate_orders_df.shape[0]}')
print(f'total unique orders with multiple rows : {duplicate_orders_df.order_id_new.nunique()}')
print(f'rows with OrderID mismatch: {len(updated_dataset[updated_dataset.order_id_mismatch_flag==1].index)}')

ordrs with multiple rows : 1389
total unique orders with multiple rows : 612
rows with OrderID mismatch: 58


In [47]:
# by the looks of it, most orders have 2 dulicate rows, plotting this to be sure

# check how many orders have multiple rows
row_count_distribution_view = updated_dataset[['order_id_new', 'total_rows_for_this_order']]
temp_view = row_count_distribution_view.groupby(['total_rows_for_this_order']).order_id_new.nunique()

print('distribution of unique orders agains row count :- ', end='\n\n')

temp_view_df = pd.DataFrame({
    'row_count_bucket': temp_view.index,
    'total_unique_orders': temp_view.values
})

print(temp_view_df.head())


distribution of unique orders agains row count :- 

   row_count_bucket  total_unique_orders
0                 1                 3554
1                 2                  487
2                 3                   94
3                 4                   24
4                 5                    5


one order can have upto 5 rows, tho most orderIds are unique.

question : what is the difference ? 


lets take a look at orders with 5 rows to figure out how each row is different from the other

In [49]:
updated_dataset[updated_dataset.total_rows_for_this_order == 5].head(10)

,order_id_new,order_try_id_new,calc_created,metered_price,upfront_price,distance,duration,gps_confidence,entered_by,b_state,dest_change_number,prediction_price_type,predicted_distance,predicted_duration,change_reason_pricing,ticket_id_new,device_token,rider_app_version,order_state,order_try_state,driver_app_version,driver_device_uid_new,device_name,eu_indicator,overpaid_ride_ticket,fraud_score,total_rows_for_this_order,total_rows,order_id_mismatch_flag
925,761,761,2020-02-09 6:51:20,20753.20,10500.0,10459,1874,1,client,finished,1,upfront,4892.0,698.0,None,2173,None,CI.4.17,finished,finished,DA.4.19,982,TECNO MOBILE LIMITED TECNO KA7O,0,0,NaN,5,4943,0
926,761,761,2020-02-09 6:51:20,20753.20,10500.0,10459,1874,1,client,finished,1,upfront,4892.0,698.0,None,2175,None,CI.4.17,finished,finished,DA.4.19,982,TECNO MOBILE LIMITED TECNO KA7O,0,0,NaN,5,4943,0
927,761,761,2020-02-09 6:51:20,20753.20,10500.0,10459,1874,1,client,finished,1,upfront,4892.0,698.0,None,529,None,CI.4.17,finished,finished,DA.4.19,982,TECNO MOBILE LIMITED TECNO KA7O,0,0,NaN,5,4943,0
928,761,761,2020-02-09 6:51:20,20753.20,10500.0,10459,1874,1,client,finished,1,upfront,4892.0,698.0,None,526,None,CI.4.17,finished,finished,DA.4.19,982,TECNO MOBILE LIMITED TECNO KA7O,0,0,NaN,5,4943,0
929,761,761,2020-02-09 6:51:20,20753.20,10500.0,10459,1874,1,client,finished,1,upfront,4892.0,698.0,None,2288,None,CI.4.17,finished,finished,DA.4.19,982,TECNO MOBILE LIMITED TECNO KA7O,0,0,NaN,5,4943,0
943,774,774,2020-02-09 10:41:45,17318.93,18000.0,12029,1864,0,client,finished,1,upfront,12469.0,1898.0,None,2230,None,CA.5.36,finished,finished,DA.4.39,1900,Alps F9 Pro,0,0,NaN,5,4943,0
944,774,774,2020-02-09 10:41:45,17318.93,18000.0,12029,1864,0,client,finished,1,upfront,12469.0,1898.0,None,3207,None,CA.5.36,finished,finished,DA.4.39,1900,Alps F9 Pro,0,0,NaN,5,4943,0
945,774,774,2020-02-09 10:41:45,17318.93,18000.0,12029,1864,0,client,finished,1,upfront,12469.0,1898.0,None,6,None,CA.5.36,finished,finished,DA.4.39,1900,Alps F9 Pro,0,0,NaN,5,4943,0
946,774,774,2020-02-09 10:41:45,17318.93,18000.0,12029,1864,0,client,finished,1,upfront,12469.0,1898.0,None,2278,None,CA.5.36,finished,finished,DA.4.39,1900,Alps F9 Pro,0,0,NaN,5,4943,0
947,774,774,2020-02-09 10:41:45,17318.93,18000.0,12029,1864,0,client,finished,1,upfront,12469.0,1898.0,None,2813,None,CA.5.36,finished,finished,DA.4.39,1900,Alps F9 Pro,0,0,NaN,5,4943,0


looks like ticket_id_new is the unique row indicator for our dataSet, i.e, most granular level of information is on customer support ticket created

lets also take a look at those orders where order-id and try-order-id are different

In [50]:
updated_dataset[updated_dataset.order_id_mismatch_flag == 1].head(5)

,order_id_new,order_try_id_new,calc_created,metered_price,upfront_price,distance,duration,gps_confidence,entered_by,b_state,dest_change_number,prediction_price_type,predicted_distance,predicted_duration,change_reason_pricing,ticket_id_new,device_token,rider_app_version,order_state,order_try_state,driver_app_version,driver_device_uid_new,device_name,eu_indicator,overpaid_ride_ticket,fraud_score,total_rows_for_this_order,total_rows,order_id_mismatch_flag
186,149,150,2020-02-03 11:18:58,17752.64,22500.0,13300,1875,1,client,finished,1,upfront,17393.0,1466.0,None,1512,None,CA.5.42,finished,finished,DA.4.37,1986,HUAWEI TRT-L21A,0,0,NaN,1,4943,1
187,150,149,2020-02-03 12:17:59,20726.77,25000.0,14339,2420,0,client,finished,1,upfront,17293.0,2203.0,None,1522,None,CA.5.19,finished,finished,DA.4.37,820,TECNO MOBILE LIMITED TECNO B1,0,0,NaN,3,4943,1
188,150,149,2020-02-03 12:17:59,20726.77,25000.0,14339,2420,0,client,finished,1,upfront,17293.0,2203.0,None,1526,None,CA.5.19,finished,finished,DA.4.37,820,TECNO MOBILE LIMITED TECNO B1,0,0,NaN,3,4943,1
189,150,149,2020-02-03 12:17:59,20726.77,25000.0,14339,2420,0,client,finished,1,upfront,17293.0,2203.0,None,1521,None,CA.5.19,finished,finished,DA.4.37,820,TECNO MOBILE LIMITED TECNO B1,0,0,NaN,3,4943,1
496,402,403,2020-02-06 6:22:57,7894.13,9000.0,4266,1000,1,client,finished,1,upfront,5439.0,1086.0,None,1918,None,CA.5.42,finished,finished,DA.4.30,514,TECNO MOBILE LIMITED TECNO LC6,0,0,NaN,1,4943,1


lets try to create a primary key for this dataset dependent on orderID

In [54]:
# question : are there orders whcich have different TryId new and also, a row with the same tryId New ?

In [66]:
query = """

with base as (

select order_id_new, 
sum(distinct case when order_id_new = order_try_id_new then 1 else 0 end) as same_id_flag,
sum(distinct case when order_id_new != order_try_id_new then 1 else 0 end) as different_id_flag
from updated_dataset
group by 1
)

select * from base


"""

In [58]:
# validate_order_id_key

order_id_view = sqldf(query)
order_id_view.head(5)


,order_id_new,same_id_flag,different_id_flag
0,0,1,0
1,1,1,0
2,2,1,0
3,3,1,0
4,4,1,0


In [65]:
print('orders with both kinds of rows : - ') 
print(len(order_id_view[(order_id_view.same_id_flag==1) & (order_id_view.different_id_flag==1)].index))
print('\n\n')
print(f'total orders where both orderIds are same: {order_id_view.same_id_flag.sum()}')
print(f'total orders where both orderIds are different: {order_id_view.different_id_flag.sum()}')

orders with both kinds of rows : - 
0



total orders where both orderIds are same: 4120
total orders where both orderIds are different: 46


In [67]:
query = """

with order_id_new as (

select order_id_new as order_id, 
sum(distinct case when order_id_new = order_try_id_new then 1 else 0 end) as same_id_flag,
sum(distinct case when order_id_new != order_try_id_new then 1 else 0 end) as different_id_flag
from updated_dataset
group by 1
),

try_new_order_id as (

select order_try_id_new as order_id,
sum(distinct case when order_id_new = order_try_id_new then 1 else 0 end) as same_id_flag,
sum(distinct case when order_id_new != order_try_id_new then 1 else 0 end) as different_id_flag
from updated_dataset
group by 1
)

select * from order_id_new
union
select * from try_new_order_id

"""

In [72]:
order_id_unique_df = sqldf(query)
print(order_id_unique_df.shape)

print('\n\n')
print(f'total orders where both orderIds are same: {order_id_unique_df.same_id_flag.sum()}')
print(f'total orders where both orderIds are different: {order_id_unique_df.different_id_flag.sum()}')

(4166, 3)



total orders where both orderIds are same: 4120
total orders where both orderIds are different: 46


there are a total of 4166 orders, so we can just use the first column as the primaryKey identifier for this analysis

basically, total unique orders in order_id_new, order_id_try_new and order_id_new Union order_id_try_new are all the same

In [52]:
updated_dataset['key'] = np.nan

In [74]:
# the problem statement is to improve the upfront pricing precision
# in order to do this, let us begin by figuring out what all orders had 
# 20%+ change in upfront and metered pricing
list(updated_dataset.columns)

['order_id_new',
 'order_try_id_new',
 'calc_created',
 'metered_price',
 'upfront_price',
 'distance',
 'duration',
 'gps_confidence',
 'entered_by',
 'b_state',
 'dest_change_number',
 'prediction_price_type',
 'predicted_distance',
 'predicted_duration',
 'change_reason_pricing',
 'ticket_id_new',
 'device_token',
 'rider_app_version',
 'order_state',
 'order_try_state',
 'driver_app_version',
 'driver_device_uid_new',
 'device_name',
 'eu_indicator',
 'overpaid_ride_ticket',
 'fraud_score',
 'total_rows_for_this_order',
 'total_rows',
 'order_id_mismatch_flag',
 'key']

In [75]:
dataset_cleaned = updated_dataset[[
 'order_id_new',
 'calc_created',
 'metered_price',
 'upfront_price',
 'distance',
 'duration',
 'gps_confidence',
 'entered_by',
 'b_state',
 'dest_change_number',
 'prediction_price_type',
 'predicted_distance',
 'predicted_duration',
 'change_reason_pricing',
 'device_token',
 'rider_app_version',
 'order_state',
 'order_try_state',
 'driver_app_version',
 'driver_device_uid_new',
 'device_name',
 'eu_indicator',
 'overpaid_ride_ticket',
 'fraud_score',
 'total_rows_for_this_order',
 'total_rows',
 'order_id_mismatch_flag',
 'key'
]].copy(deep=True)

In [89]:
dataset_cleaned.drop(['key', 'total_rows_for_this_order', 'total_rows', 'order_id_mismatch_flag'], axis=1, inplace=True)
dataset_cleaned.head(5)

,order_id_new,calc_created,metered_price,upfront_price,distance,duration,gps_confidence,entered_by,b_state,dest_change_number,prediction_price_type,predicted_distance,predicted_duration,change_reason_pricing,device_token,rider_app_version,order_state,order_try_state,driver_app_version,driver_device_uid_new,device_name,eu_indicator,overpaid_ride_ticket,fraud_score
0,0,2020-02-02 0:01:16,7.53,4.5,6153,1194,1,client,finished,1,upfront,4379.0,480.0,None,None,CA.5.42,finished,finished,DA.4.30,637,HMD Global TA-1032,1,0,NaN
1,1,2020-02-02 0:11:53,2.50,3.5,119,91,1,client,finished,1,upfront,3906.0,444.0,None,None,CA.5.42,finished,finished,DA.4.34,1370,Xiaomi Redmi 5,1,0,-41.0
2,2,2020-02-02 0:22:05,4.09,NaN,2523,374,1,driver,finished,2,upfront_destination_changed,10570.0,1040.0,driver_destination_changed,None,CA.5.42,finished,finished,DA.4.37,1629,Samsung SM-G955F,1,0,NaN
3,3,2020-02-02 0:49:24,14.87,NaN,15541,1690,0,client,finished,2,upfront_destination_changed,5647.0,697.0,client_destination_changed,None,CI.4.17,finished,finished,DA.4.37,1257,Samsung SM-J415FN,1,0,-1516.0
5,4,2020-02-02 1:12:00,29.16,26.8,51267,2809,1,client,finished,1,upfront,46059.0,2723.0,None,None,CA.5.42,finished,finished,DA.4.37,1079,Samsung SM-G903W,1,0,-2.0


In [84]:
print(f'total rows : {len(dataset_cleaned.index)}')
print(f'total orders unique : {dataset_cleaned.order_id_new.nunique()}')
dataset_cleaned.drop_duplicates(inplace=True)
print(f'total rows after removing duplicate rows : {len(dataset_cleaned.index)}')      

total rows : 4270
total orders unique : 4166
total rows after removing duplicate rows : 4270


In [86]:
# okay, contrary to earlier assumption, there are 
# nonDuplicate rows even tho we have removed the ticket-id feild
# lets figure out why this may be

In [87]:
# dataset_cleaned['row_count_for_order'] = np.nan

In [90]:
total_row_df = dataset_cleaned.copy(deep=True)
total_row_df = dataset_cleaned.groupby(['order_id_new']).order_id_new.count()

In [ ]:
# converting the series to df again
total_row_df = pd.DataFrame({
    'order_id_new': total_row_df.index,
    'total_rows': total_row_df.values
})

In [101]:
dataset_cleaned = dataset_cleaned.merge(total_row_df, how='left', on='order_id_new')

In [102]:
dataset_cleaned.head(2)

,order_id_new,calc_created,metered_price,upfront_price,distance,duration,gps_confidence,entered_by,b_state,dest_change_number,prediction_price_type,predicted_distance,predicted_duration,change_reason_pricing,device_token,rider_app_version,order_state,order_try_state,driver_app_version,driver_device_uid_new,device_name,eu_indicator,overpaid_ride_ticket,fraud_score,total_rows
0,0,2020-02-02 0:01:16,7.53,4.5,6153,1194,1,client,finished,1,upfront,4379.0,480.0,None,None,CA.5.42,finished,finished,DA.4.30,637,HMD Global TA-1032,1,0,NaN,1
1,1,2020-02-02 0:11:53,2.50,3.5,119,91,1,client,finished,1,upfront,3906.0,444.0,None,None,CA.5.42,finished,finished,DA.4.34,1370,Xiaomi Redmi 5,1,0,-41.0,1


In [103]:
dataset_cleaned[dataset_cleaned.total_rows>1].head(10)

,order_id_new,calc_created,metered_price,upfront_price,distance,duration,gps_confidence,entered_by,b_state,dest_change_number,prediction_price_type,predicted_distance,predicted_duration,change_reason_pricing,device_token,rider_app_version,order_state,order_try_state,driver_app_version,driver_device_uid_new,device_name,eu_indicator,overpaid_ride_ticket,fraud_score,total_rows
63,63,2020-02-02 14:25:38,149723.20,23000.0,53256,1111,0,client,finished,1,upfront,12712.0,1843.0,None,None,CA.5.42,finished,finished,DA.4.10,138,HUAWEI FRD-L09,0,0,NaN,2
64,63,2020-02-02 14:25:38,149723.20,23000.0,53256,1111,0,client,finished,1,upfront,12712.0,1843.0,None,None,CA.5.42,finished,finished,DA.4.10,138,HUAWEI FRD-L09,0,1,NaN,2
128,127,2020-02-03 6:19:13,8484.04,6000.0,5738,571,0,client,finished,1,upfront,2913.0,670.0,None,None,CA.4.89,finished,finished,DA.4.31,290,Samsung GT-I9301I,0,1,NaN,2
129,127,2020-02-03 6:19:13,8484.04,6000.0,5738,571,0,client,finished,1,upfront,2913.0,670.0,None,None,CA.4.89,finished,finished,DA.4.31,290,Samsung GT-I9301I,0,0,NaN,2
203,201,2020-02-03 21:46:30,18929.92,6500.0,14560,1421,0,client,finished,1,upfront,3789.0,538.0,None,None,CA.4.99,finished,finished,DA.4.10,17,TECNO MOBILE LIMITED TECNO B1p,0,1,NaN,2
204,201,2020-02-03 21:46:30,18929.92,6500.0,14560,1421,0,client,finished,1,upfront,3789.0,538.0,None,None,CA.4.99,finished,finished,DA.4.10,17,TECNO MOBILE LIMITED TECNO B1p,0,0,NaN,2
228,225,2020-02-04 9:26:05,9788.40,5000.0,15396,2206,1,client,finished,1,upfront,7813.0,1288.0,None,None,CA.5.42,finished,finished,DA.4.37,1862,TECNO F3,0,1,NaN,2
229,225,2020-02-04 9:26:05,9788.40,5000.0,15396,2206,1,client,finished,1,upfront,7813.0,1288.0,None,None,CA.5.42,finished,finished,DA.4.37,1862,TECNO F3,0,0,NaN,2
312,308,2020-02-05 7:29:06,45932.22,17500.0,23576,9391,0,client,finished,1,upfront,13077.0,2267.0,None,None,CI.4.17,finished,finished,DA.4.30,538,Samsung SM-G610F,0,1,NaN,2
313,308,2020-02-05 7:29:06,45932.22,17500.0,23576,9391,0,client,finished,1,upfront,13077.0,2267.0,None,None,CI.4.17,finished,finished,DA.4.30,538,Samsung SM-G610F,0,0,NaN,2


looks like the difference is caused by multiple values for overpaid_tide_ticket
lets clean this up and see if the data checks out


In [114]:
# Note : the distinct sum will make overpaid_ride_ticket in the following query work like a flag
# if, for any particular ticket-id for an order, the user says overpaid, 
# then then we can flatten the table by marking it 1

query = """

with base as (

select order_id_new as order_id, 
calc_created, 
metered_price, upfront_price,
distance, duration, gps_confidence, 
entered_by, b_state, dest_change_number, 
prediction_price_type,
predicted_distance, predicted_duration, 
change_reason_pricing, rider_app_version, 
order_state, driver_app_version, driver_device_uid_new,
device_name, eu_indicator, fraud_score, 

sum(distinct overpaid_ride_ticket) as overpaid_ride_ticket

from dataset_cleaned
group by 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
         13, 14, 15, 16, 17, 18, 19, 20, 21

)

select * from base

"""

In [109]:
main_table = sqldf(query)
main_table.head(2)

,order_id,calc_created,metered_price,upfront_price,distance,duration,gps_confidence,entered_by,b_state,dest_change_number,prediction_price_type,predicted_distance,predicted_duration,change_reason_pricing,rider_app_version,order_state,driver_app_version,driver_device_uid_new,device_name,eu_indicator,fraud_score,overpaid_ride_ticket
0,0,2020-02-02 0:01:16,7.53,4.5,6153,1194,1,client,finished,1,upfront,4379.0,480.0,None,CA.5.42,finished,DA.4.30,637,HMD Global TA-1032,1,NaN,0
1,1,2020-02-02 0:11:53,2.50,3.5,119,91,1,client,finished,1,upfront,3906.0,444.0,None,CA.5.42,finished,DA.4.34,1370,Xiaomi Redmi 5,1,-41.0,0


In [110]:
print(f'total rows : {len(main_table.index)}')
print(f'unique orders : {main_table.order_id.nunique()}')

total rows : 4166
unique orders : 4166


now, we have 4166 rows, i.e, the table is flat on an orderID level

we can finally start working on the problem statement now

#### problem statement : predictions are considered off if there is a difference of more than 20% between metered and upfront price

in order to find out how many such cases exist, we first need to figure out how many such cases exist, and then look for any features that cause this

In [120]:
# question : are there cases where either of the prices are null ?
print(f'null upfront price count : {main_table.upfront_price.isna().sum()}')
print(f'null metered price count : {main_table.metered_price.isna().sum()}')
print(f'null price for both : {main_table[(main_table.metered_price.isnull()) & (main_table.upfront_price.isnull())].order_id.count()}')

null upfront price count : 1220
null metered price count : 19
null price for both : 19


In [135]:
# so, cases where metered price is null, upfront price is also null. 
# we cant improve what we dont know, so we will drop these 19 orders
# OR
# because we have multiple rows with upfront-price, distance and duration, 

# we can try fioguring out the wieghts by using something like :
# w_distance = metered_price/distance
# w_duration = metered_price/duration

In [136]:
# checking if weights actually work this way 
test_cases = {
    'case1': {
        'metered_price': 4.04,
        'distance': 2839,
        'duration': 700
    },
    'case2': {
        'metered_price': 6.09,
        'distance': 5698,
        'duration': 493
    },
    'case3': {
        'metered_price': 4.32,
        'distance': 4426,
        'duration': 695
    }
}

def check_ratio(test_cases):
    for key, value in test_cases.items():
        metered_price = value.get('metered_price')
        distance = value.get('distance')
        duration = value.get('duration')

        # let x be the rati in which price depends on weight and distance
        w_distance = metered_price/distance
        w_duration = metered_price/duration
        print(f'meteredPrice, distance, duration, w_distance, w_duration : {metered_price}, {distance}, {duration}, {w_distance}, {w_duration}')

In [137]:
check_ratio(test_cases=test_cases)

meteredPrice, distance, duration, w_distance, w_duration : 4.04, 2839, 700, 0.0014230362803804156, 0.0057714285714285714
meteredPrice, distance, duration, w_distance, w_duration : 6.09, 5698, 493, 0.0010687960687960688, 0.012352941176470587
meteredPrice, distance, duration, w_distance, w_duration : 4.32, 4426, 695, 0.0009760506100316313, 0.006215827338129497


ratio values are different, hence, we can not use this formula to fill in Null metered_prices

hence, we will drop those rows

In [141]:
main_table.dropna(subset=['metered_price'], axis=0, how='any', inplace=True)
main_table.order_id.count() 
# should be 4166 - 19 = 4147 
# [ 19 is rows with neither predicted nor actual price]

4147

checks out

In [142]:
# for cases where we have upfront price, but no meter price, we will assume 
# that upfront price was the same as metered price

In [143]:
# now, we will add two columns to the table, 
# 1. prediction_wrong_by_percentage : by what percentage was the prediction off ? 
# 2. bad_prediction_flag --> if prediction was off by 20 percent, this will be 1

In [145]:
def bad_prediction_flag(row):
    """
    update values for the following columns in the row:
    
    prediction_wrong_by_percentage: % delta between predicted and metered price
    bad_prediction_flag: 1 if prediction is off by more than 20%
    
    note : dataSet still has cases where upfront price is null
    """
    upfront_price = row.upfront_price
    metered_price = row.metered_price
    
    # find the percentage change from upfront_to_metered 
    percentage_change = metered_price*100.00/upfront_price - 100.00
    if percentage_change == np.nan:
        # this will be true when upfront price is null
        row.prediction_wrong_by_percentage = 0
    else:
        row.prediction_wrong_by_percentage = round(percentage_change, 2)
                                       # return 0 for null values
    
    # flag = 1 if prediction is wrong
    if abs(percentage_change) >= 20:
        row.bad_prediction_flag = 1
    else:
        row.bad_prediction_flag = 0
    
    return row

In [146]:
main_table['prediction_wrong_by_percentage'] = np.nan
main_table['bad_prediction_flag'] = np.nan

In [148]:
# pass in function to transform each row
main_table = main_table.apply(bad_prediction_flag, axis='columns') 

In [149]:
main_table.head()

,order_id,calc_created,metered_price,upfront_price,distance,duration,gps_confidence,entered_by,b_state,dest_change_number,prediction_price_type,predicted_distance,predicted_duration,change_reason_pricing,rider_app_version,order_state,driver_app_version,driver_device_uid_new,device_name,eu_indicator,fraud_score,overpaid_ride_ticket,prediction_wrong_by_percentage,bad_prediction_flag
0,0,2020-02-02 0:01:16,7.53,4.5,6153,1194,1,client,finished,1,upfront,4379.0,480.0,None,CA.5.42,finished,DA.4.30,637,HMD Global TA-1032,1,NaN,0,67.33,1
1,1,2020-02-02 0:11:53,2.50,3.5,119,91,1,client,finished,1,upfront,3906.0,444.0,None,CA.5.42,finished,DA.4.34,1370,Xiaomi Redmi 5,1,-41.0,0,-28.57,1
2,2,2020-02-02 0:22:05,4.09,NaN,2523,374,1,driver,finished,2,upfront_destination_changed,10570.0,1040.0,driver_destination_changed,CA.5.42,finished,DA.4.37,1629,Samsung SM-G955F,1,NaN,0,NaN,0
3,3,2020-02-02 0:49:24,14.87,NaN,15541,1690,0,client,finished,2,upfront_destination_changed,5647.0,697.0,client_destination_changed,CI.4.17,finished,DA.4.37,1257,Samsung SM-J415FN,1,-1516.0,0,NaN,0
4,4,2020-02-02 1:12:00,29.16,26.8,51267,2809,1,client,finished,1,upfront,46059.0,2723.0,None,CA.5.42,finished,DA.4.37,1079,Samsung SM-G903W,1,-2.0,0,8.81,0


In [152]:
# okay, now, to answer some questions

In [158]:
# what percentage of orders have wrong predictions ?
bad_prediction_orders = main_table[main_table.bad_prediction_flag == 1].order_id.count()
print(f'bad prediction orders : {bad_prediction_orders}')
total_orders = main_table.order_id.count()
print(f'total orders : {total_orders}')
print(f'percentage of incorrectly predicted orders : {round(bad_prediction_orders*100.00/total_orders, 2)}')

bad prediction orders : 1352
total orders : 4147
percentage of incorrectly predicted orders : 32.6


In [160]:
# so, our job here is to reduce this count, and get the prediction accuracy above 67%

In [179]:
# feilds that seem interesting and possibly have some sort of relationship with the prediction
# gps_confidence, entered_by, dest_change_number, prediction_price_type, change_reason_pricing
# fraud_score

## GPS Confidence 

In [165]:
good_gps_orders = main_table[main_table.gps_confidence == 1].order_id.count()
print(f'orders with goodGps confidence : {good_gps_orders}')
print(f'percentage of orders with goodGPS confidence: {round((good_gps_orders*100.00/total_orders), 2)}')

orders with goodGps confidence : 3387
percentage of orders with goodGPS confidence: 81.67


In [172]:
# check prediction dependence on gps score
good_gps_and_correct_prediction = main_table[(main_table.gps_confidence == 1) & (main_table.bad_prediction_flag == 0)].order_id.count()
good_gps_wrong_prediction = main_table[(main_table.gps_confidence == 1) & (main_table.bad_prediction_flag == 1)].order_id.count()
bad_gps_correct_prediction = main_table[(main_table.gps_confidence == 0) & (main_table.bad_prediction_flag == 0)].order_id.count()
bad_gps_wrong_prediction = main_table[(main_table.gps_confidence == 0) & (main_table.bad_prediction_flag == 1)].order_id.count()

print(f'total orders with good gps confidence and correct prediction : {good_gps_and_correct_prediction}')
print(f'perentage : {round(good_gps_and_correct_prediction*100.00/total_orders)}')
print(f'total orders with good gps and wrong prediction : {good_gps_wrong_prediction}')
print(f'perentage : {round(good_gps_wrong_prediction*100.00/total_orders)}')
print(f'total orders with bad gps and correct prediction : {bad_gps_correct_prediction}')
print(f'perentage : {round(bad_gps_correct_prediction*100.00/total_orders)}')
print(f'total orders with bad gps and correct prediction : {bad_gps_wrong_prediction}')
print(f'perentage : {round(bad_gps_wrong_prediction*100.00/total_orders)}')


total orders with good gps confidence and correct prediction : 2242
perentage : 54
total orders with good gps and wrong prediction : 1145
perentage : 28
total orders with bad gps and correct prediction : 553
perentage : 13
total orders with bad gps and correct prediction : 207
perentage : 5


82 percent of orders have good gps confidence, 54 % of which are accurate price predictions, 
and 28 % price predictions are off

conclusion : gps confidence score does not heavily influence price

## entered By

In [177]:
main_table.entered_by.unique()

array(['client', 'driver'], dtype=object)

In [262]:
# check prediction dependence on entered by score
client_correct_prediction = main_table[(main_table.entered_by == 'client') & (main_table.bad_prediction_flag == 0)].order_id.count()
client_wrong_prediction = main_table[(main_table.entered_by == 'client') & (main_table.bad_prediction_flag == 1)].order_id.count()
driver_correct_prediction = main_table[(main_table.entered_by == 'driver') & (main_table.bad_prediction_flag == 0)].order_id.count()
driver_wrong_prediction = main_table[(main_table.entered_by == 'driver') & (main_table.bad_prediction_flag == 1)].order_id.count()

print(f'entered by client and correct prediction : {client_correct_prediction}')
print(f'perentage : {round(client_correct_prediction*100.00/total_orders)}')
print(f'entered by client and wrong prediction : {client_wrong_prediction}')
print(f'perentage : {round(client_wrong_prediction*100.00/total_orders)}')
print(f'entered by driver and correct prediction : {bad_gps_correct_prediction}')
print(f'perentage : {round(driver_correct_prediction*100.00/total_orders)}')
print(f'entered by driver and wrong prediction : {bad_gps_wrong_prediction}')
print(f'perentage : {round(driver_wrong_prediction*100.00/total_orders)}')


entered by client and correct prediction : 2645
perentage : 64
entered by client and wrong prediction : 1325
perentage : 32
entered by driver and correct prediction : 553
perentage : 4
entered by driver and wrong prediction : 207
perentage : 1


90 percent of destinations are entered by clients, 60 percent are accurate, and 30 percent are not

 ## destination changes

In [182]:
dest_not_change_correct_prediction = main_table[(main_table.dest_change_number == 1) & (main_table.bad_prediction_flag == 0)].order_id.count()
dest_not_change_wrong_prediction = main_table[(main_table.dest_change_number == 1) & (main_table.bad_prediction_flag == 1)].order_id.count()
dest_change_correct_prediction = main_table[(main_table.dest_change_number > 1) & (main_table.bad_prediction_flag == 0)].order_id.count()
dest_change_wrong_prediction = main_table[(main_table.dest_change_number > 1) & (main_table.bad_prediction_flag == 1)].order_id.count()

print(f'destination not changed and correct prediction : {dest_not_change_correct_prediction}')
print(f'perentage : {round(dest_not_change_correct_prediction*100.00/total_orders)}')
print(f'destination not changed and wrong prediction : {dest_not_change_wrong_prediction}')
print(f'perentage : {round(dest_not_change_wrong_prediction*100.00/total_orders)}')
print(f'destination changed and correct prediction : {dest_change_correct_prediction}')
print(f'perentage : {round(dest_change_correct_prediction*100.00/total_orders)}')
print(f'destination changed and correct prediction : {dest_change_wrong_prediction}')
print(f'perentage : {round(dest_change_wrong_prediction*100.00/total_orders)}')


destination not changed and correct prediction : 2528
perentage : 61
destination not changed and wrong prediction : 1285
perentage : 31
destination changed and correct prediction : 267
perentage : 6
destination changed and correct prediction : 67
perentage : 2


destination change does not look like a reliable metric to improve prediction accuracy

## prediction price type

In [186]:
main_table.prediction_price_type.unique()

array(['upfront', 'upfront_destination_changed',
       'upfront_waypoint_changed', 'prediction'], dtype=object)

In [187]:
upfront_correct_prediction = main_table[(main_table.prediction_price_type == 'upfront') & (main_table.bad_prediction_flag == 0)].order_id.count()
upfront_wrong_prediction = main_table[(main_table.prediction_price_type == 'upfront') & (main_table.bad_prediction_flag == 1)].order_id.count()

upfront_dest_changed_correct_prediction = main_table[(main_table.prediction_price_type == 'upfront_destination_changed') & (main_table.bad_prediction_flag == 0)].order_id.count()
upfront_dest_changed_wrong_prediction = main_table[(main_table.prediction_price_type == 'upfront_destination_changed') & (main_table.bad_prediction_flag == 1)].order_id.count()

upfront_waypoint_changed_correct_prediction = main_table[(main_table.prediction_price_type == 'upfront_waypoint_changed') & (main_table.bad_prediction_flag == 0)].order_id.count()
upfront_waypoint_changed_wrong_prediction = main_table[(main_table.prediction_price_type == 'upfront_waypoint_changed') & (main_table.bad_prediction_flag == 1)].order_id.count()

prediction_correct_prediction = main_table[(main_table.prediction_price_type == 'prediction') & (main_table.bad_prediction_flag == 0)].order_id.count()
prediction_wrong__prediction = main_table[(main_table.prediction_price_type == 'prediction') & (main_table.bad_prediction_flag == 1)].order_id.count()



print(f'UPFRONT and correct prediction : {upfront_correct_prediction}')
print(f'perentage : {round(upfront_correct_prediction*100.00/total_orders)}')

print(f'UPFRONT and wrong prediction : {upfront_wrong_prediction}')
print(f'perentage : {round(upfront_wrong_prediction*100.00/total_orders)}')

print(f'UPFRONT DEST changed and correct prediction : {upfront_dest_changed_correct_prediction}')
print(f'perentage : {round(upfront_dest_changed_correct_prediction*100.00/total_orders)}')

print(f'UPFRONT DEST changed and wrong prediction : {upfront_dest_changed_wrong_prediction}')
print(f'perentage : {round(upfront_dest_changed_wrong_prediction*100.00/total_orders)}')

print(f'UPFRONT WAYPOINT changed and correct prediction : {upfront_waypoint_changed_correct_prediction}')
print(f'perentage : {round(upfront_waypoint_changed_correct_prediction*100.00/total_orders)}')

print(f'UPFRONT WAYPOINT and wrong prediction : {upfront_waypoint_changed_wrong_prediction}')
print(f'perentage : {round(upfront_waypoint_changed_wrong_prediction*100.00/total_orders)}')

print(f'PREDICTION and correct prediction : {prediction_correct_prediction}')
print(f'perentage : {round(prediction_correct_prediction*100.00/total_orders)}')

print(f'PREDICTION and wrong prediction : {prediction_wrong__prediction}')
print(f'perentage : {round(prediction_wrong__prediction*100.00/total_orders)}')



UPFRONT and correct prediction : 1615
perentage : 39
UPFRONT and wrong prediction : 1352
perentage : 33
UPFRONT DEST changed and correct prediction : 179
perentage : 4
UPFRONT DEST changed and wrong prediction : 0
perentage : 0
UPFRONT WAYPOINT changed and correct prediction : 3
perentage : 0
UPFRONT WAYPOINT and wrong prediction : 0
perentage : 0
PREDICTION and correct prediction : 998
perentage : 24
PREDICTION and wrong prediction : 0
perentage : 0


amidst types of prediction, for prediction that happened before the ride

In [198]:
UPFRONT_accuracy = 1615*100/(1615+1352)
PREDICTION_accuracy = 988*100/988

print(f'accuracy when type is UPFRONT: {round(UPFRONT_accuracy)}%')
print(f'accuracy when type is PREDICTION : {(PREDICTION_accuracy)}%')

accuracy when type is UPFRONT: 54%
accuracy when type is PREDICTION : 100.0%


Looks like internal variable used for pricing prediction is most accurate when prediction_price_type = prediction

for the dataset we have in hand, the accuracy is as high as 100%, while for prediction_price_type = upfront, accuracy is ~54%

as there are a significant number of orders from our dataset -> 1615+1352+988 = 3955 out of 4166, which is ~95 percent. 

`conclusion`

for most orders, price prediction happens upfront, i.e, dest_change_number = 92% in the given dataset. For cases where there are no destination changes, price prediction needs to happen only once, which is before the ride

type: Upfront : prediction correctness accuracy is 54%
type: Prediction : accuracy is 100%

based on the information at hand, we can assume that `price_prediction_type = prediction` is more accurate

if all of these orderes, price prediction had been of type prediction, 
accuracy would be : (1615+1352+988)/4147 = 95%

so, we would have gotten down the predicition inaccuracy from 32 percent to ~5% percent,
which is pretty coool

note : 
1. this is based on the assumption that there is a different price prediction logic that is used for both of these types
2. there is also the fact that its unlikely that the prediction accuracy is actually 100 percent, and we should look at more data to draw a more accurate precision figure

In [201]:
(1615+1352+988)/4147

0.9537014709428503

In [200]:
total_orders

4147

### some other rough calculations to add to the business pdf

In [208]:
main_table.calc_created.astype('datetime64[D]').describe()

/var/folders/ns/bqgvlxhs2f94gqgtggjtsvm00000gn/T/ipykernel_6188/2005184764.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  main_table.calc_created.astype('datetime64[D]').describe()


count                    4147
unique                     41
top       2020-02-14 00:00:00
freq                      157
first     2020-02-02 00:00:00
last      2020-03-13 00:00:00
Name: calc_created, dtype: object

In [242]:
# checking if there is any relationship between prediction_type and upfront_price being null

view = main_table[['order_id', 'prediction_price_type', 'upfront_price']].copy(deep=True)

view['upfront_price_isnull'] = np.nan
print(view.head(5))

   order_id        prediction_price_type  upfront_price  upfront_price_isnull
0         0                      upfront            4.5                   NaN
1         1                      upfront            3.5                   NaN
2         2  upfront_destination_changed            NaN                   NaN
3         3  upfront_destination_changed            NaN                   NaN
4         4                      upfront           26.8                   NaN


In [256]:
def upfront_price_makebucket(row):
    """
    if upfront price is >=0, then isNull flag = 0
    else, isNull flag = 1
    """
    
    upfront_price = row.get('upfront_price')
    
    if upfront_price >= 0:
        row.upfront_price_isnull = 0
    else:
        row.upfront_price = 0
        row.upfront_price_isnull = 1
        
    return row

In [254]:
view = view.apply(upfront_price_makebucket, axis='columns')

In [255]:
view.head()

,order_id,prediction_price_type,upfront_price,upfront_price_isnull
0,0,upfront,4.5,0
1,1,upfront,3.5,0
2,2,upfront_destination_changed,0.0,1
3,3,upfront_destination_changed,0.0,1
4,4,upfront,26.8,0


In [257]:
view.upfront_price_isnull.unique()

array([0, 1])

In [258]:
view.groupby(['prediction_price_type']).upfront_price_isnull.sum()

prediction_price_type
prediction                     998
upfront                         21
upfront_destination_changed    179
upfront_waypoint_changed         3
Name: upfront_price_isnull, dtype: int64

In [259]:
# looks like all cases where prediction price type is : prediction is NULL

In [261]:
# makes sense why the prediction accuracy was 100 percent for these

In [264]:
main_table.rider_app_version.unique()

array(['CA.5.42', 'CI.4.17', 'CI.4.11', 'CI.4.14', 'CA.5.32', 'CA.5.40',
       'CI.4.00', 'CA.5.36', 'CA.5.38', 'CA.5.30', 'CI.4.13', 'CA.5.26',
       'CA.4.75', 'CA.4.77', 'CI.4.06', 'CA.4.52', 'CA.4.73', 'CA.4.91',
       'CI.4.07', 'CI.4.16', 'CA.5.04', 'CA.5.23', 'CA.4.81', 'CA.4.89',
       'CA.5.27', 'CA.5.19', 'CI.3.74', 'CA.5.08', 'CI.4.01', 'CA.4.41',
       'CI.4.04', 'CA.4.93', 'CI.4.02', 'CA.4.99', 'CA.5.17', 'CI.4.10',
       'CA.5.02', 'CA.4.66', 'CA.4.43', 'CA.5.01', 'CI.4.03', 'CA.4.79',
       'CA.4.72', 'CA.5.43', 'CI.4.12', 'CA.5.07', 'CA.5.00', 'CA.5.37',
       'CI.3.99', 'CI.3.77', 'CA.5.13', 'CA.4.87', 'CA.4.45', 'CI.3.93',
       'CI.4.09', 'CA.4.71', 'CI.3.78', 'CA.4.97', 'CA.5.21', 'CI.3.91',
       'CA.4.86', 'CA.4.82', 'CI.3.67', 'CA.5.25', 'CI.3.84', 'CA.5.35',
       'CA.4.60', 'CA.4.95', 'CI.4.18', 'CA.5.22', 'CI.3.62', 'CI.3.79',
       'CI.3.94', 'CA.4.27', 'CA.5.44', 'CI.3.61', 'CA.4.44', 'CI.3.92',
       'CA.4.83', 'CI.3.58', 'CA.4.67', 'CI.3.96', 

In [273]:
# Are the older app versions less accurate at predicting the Distance ?

# Does CA or CI have any imact on the prediction ?

# main_table['user_app_type'] = np.nan
# main_table['user_app_version'] = np.nan

main_table['user_app_type'] = main_table.rider_app_version.str.split('.')
main_table['user_app_version'] = main_table.rider_app_version.str.split('.')


In [274]:
main_table.head(2)

,order_id,calc_created,metered_price,upfront_price,distance,duration,gps_confidence,entered_by,b_state,dest_change_number,prediction_price_type,predicted_distance,predicted_duration,change_reason_pricing,rider_app_version,order_state,driver_app_version,driver_device_uid_new,device_name,eu_indicator,fraud_score,overpaid_ride_ticket,prediction_wrong_by_percentage,bad_prediction_flag,user_app_type,user_app_version
0,0,2020-02-02 0:01:16,7.53,4.5,6153,1194,1,client,finished,1,upfront,4379.0,480.0,None,CA.5.42,finished,DA.4.30,637,HMD Global TA-1032,1,NaN,0,67.33,1,"[CA, 5, 42]","[CA, 5, 42]"
1,1,2020-02-02 0:11:53,2.50,3.5,119,91,1,client,finished,1,upfront,3906.0,444.0,None,CA.5.42,finished,DA.4.34,1370,Xiaomi Redmi 5,1,-41.0,0,-28.57,1,"[CA, 5, 42]","[CA, 5, 42]"


In [277]:
main_table.user_app_type = main_table.user_app_type.map(lambda x: x[0])
main_table.user_app_version = main_table.user_app_version.map(lambda x: x[1])

In [278]:
main_table.head(2)

,order_id,calc_created,metered_price,upfront_price,distance,duration,gps_confidence,entered_by,b_state,dest_change_number,prediction_price_type,predicted_distance,predicted_duration,change_reason_pricing,rider_app_version,order_state,driver_app_version,driver_device_uid_new,device_name,eu_indicator,fraud_score,overpaid_ride_ticket,prediction_wrong_by_percentage,bad_prediction_flag,user_app_type,user_app_version
0,0,2020-02-02 0:01:16,7.53,4.5,6153,1194,1,client,finished,1,upfront,4379.0,480.0,None,CA.5.42,finished,DA.4.30,637,HMD Global TA-1032,1,NaN,0,67.33,1,CA,5
1,1,2020-02-02 0:11:53,2.50,3.5,119,91,1,client,finished,1,upfront,3906.0,444.0,None,CA.5.42,finished,DA.4.34,1370,Xiaomi Redmi 5,1,-41.0,0,-28.57,1,CA,5


In [279]:
# Now, lets check if one app version is more accurate than the other

In [281]:
app_type_view = main_table[['order_id', 'bad_prediction_flag', 'user_app_type']].copy(deep=True)

In [282]:
query = """
with base as (

select 
user_app_type,
sum(bad_prediction_flag) as total_bad_prediction_orders,
count(order_id) as total_orders
from app_type_view
group by 1
)

select user_app_type, total_bad_prediction_orders, total_orders,
total_bad_prediction_orders*100.00/total_orders as bad_prediction_percentage
from base

"""

In [283]:
app_type_view_df = sqldf(query)

In [284]:
app_type_view_df.head()

,user_app_type,total_bad_prediction_orders,total_orders,bad_prediction_percentage
0,CA,713,2326,30.653482
1,CI,639,1821,35.090610


In [285]:
app_version_view = main_table[['order_id', 'bad_prediction_flag', 'user_app_type', 'user_app_version']].copy(deep=True)

In [288]:
query = """
with base as (

select 
user_app_type, user_app_version,
sum(bad_prediction_flag) as total_bad_prediction_orders,
count(order_id) as total_orders
from app_version_view
group by 1, 2
)

select user_app_type, user_app_version, total_bad_prediction_orders, total_orders,
total_bad_prediction_orders*100.00/total_orders as bad_prediction_percentage
from base

"""

In [289]:
app_version_view_df = sqldf(query)

In [291]:
app_version_view_df.shape

(4, 5)

In [292]:
app_version_view_df.head()

,user_app_type,user_app_version,total_bad_prediction_orders,total_orders,bad_prediction_percentage
0,CA,4,34,116,29.310345
1,CA,5,679,2210,30.723982
2,CI,3,14,51,27.450980
3,CI,4,625,1770,35.310734


In [293]:
# adoption in newer app versions looks higher, but the prediction is not better
# between riderApp type CA and CI, CA seems to have better prediction accuracy by 5 percentage points

# might be something worth looking into

### Fraud Score

In [297]:
main_table.fraud_score.describe()

count     1926.000000
mean      -679.034787
std       1117.813487
min     -14225.000000
25%       -835.750000
50%       -280.000000
75%        -63.250000
max         49.000000
Name: fraud_score, dtype: float64

In [298]:
# okay, so most fraud values are below 0
#  we will assume that cases where fraud_score > 0 are bad actors, 
# and replace null values with zeros

In [300]:
main_table.fraud_score.fillna(value=0, inplace=True)

In [301]:
main_table.head(5)

,order_id,calc_created,metered_price,upfront_price,distance,duration,gps_confidence,entered_by,b_state,dest_change_number,prediction_price_type,predicted_distance,predicted_duration,change_reason_pricing,rider_app_version,order_state,driver_app_version,driver_device_uid_new,device_name,eu_indicator,fraud_score,overpaid_ride_ticket,prediction_wrong_by_percentage,bad_prediction_flag,user_app_type,user_app_version
0,0,2020-02-02 0:01:16,7.53,4.5,6153,1194,1,client,finished,1,upfront,4379.0,480.0,None,CA.5.42,finished,DA.4.30,637,HMD Global TA-1032,1,0.0,0,67.33,1,CA,5
1,1,2020-02-02 0:11:53,2.50,3.5,119,91,1,client,finished,1,upfront,3906.0,444.0,None,CA.5.42,finished,DA.4.34,1370,Xiaomi Redmi 5,1,-41.0,0,-28.57,1,CA,5
2,2,2020-02-02 0:22:05,4.09,NaN,2523,374,1,driver,finished,2,upfront_destination_changed,10570.0,1040.0,driver_destination_changed,CA.5.42,finished,DA.4.37,1629,Samsung SM-G955F,1,0.0,0,NaN,0,CA,5
3,3,2020-02-02 0:49:24,14.87,NaN,15541,1690,0,client,finished,2,upfront_destination_changed,5647.0,697.0,client_destination_changed,CI.4.17,finished,DA.4.37,1257,Samsung SM-J415FN,1,-1516.0,0,NaN,0,CI,4
4,4,2020-02-02 1:12:00,29.16,26.8,51267,2809,1,client,finished,1,upfront,46059.0,2723.0,None,CA.5.42,finished,DA.4.37,1079,Samsung SM-G903W,1,-2.0,0,8.81,0,CA,5


In [304]:
# how many orders do we have with fraud drivers ?

fraud_orders = main_table[main_table.fraud_score > 0].order_id.count()
fraud_orders_with_bad_prediction = main_table[(main_table.fraud_score > 0) & (main_table.bad_prediction_flag > 0)].order_id.count()

print('fraud orders : ' + str(fraud_orders))
print('fraud orders with bad prediction : ' + str(fraud_orders_with_bad_prediction))
print(f'bad prediction attributable to fraud orders percentage : {fraud_orders_with_bad_prediction*100.00/fraud_orders}')

fraud orders : 41
fraud orders with bad prediction : 7
bad prediction attributable to fraud orders percentage : 17.073170731707318


In [305]:
# does not seem like fraud orders are contributing heavily to inaccurate predictions either

In [307]:
ls -a

./
../
.DS_Store
.git/
.ipynb_checkpoints/
Candidate?s Hiring Pack. Product Analytics @ Bolt.pdf
Promblem1.pdf
exploratory-analysis.ipynb
exploring-test-csv.ipynb
problem1-data-analysis.ipynb
test.csv
